In [1]:
#!/usr/bin/env python3
import argparse
import numpy as nps
from ase import Atoms
from ase.io import read, write
from ase.optimize import *
import torch
from spo.spookynet import SpookyNet
"""
Spookynet: Learning force fields with electronic degrees of freedom and nonlocal effects.
OT Unke, S Chmiela, M Gastegger, KT Schütt, HE Sauceda, KR Müller
Nat. Commun. 12(1), 2021, 1-14, (2021).
"""


'\nSpookynet: Learning force fields with electronic degrees of freedom and nonlocal effects.\nOT Unke, S Chmiela, M Gastegger, KT Schütt, HE Sauceda, KR Müller\nNat. Commun. 12(1), 2021, 1-14, (2021).\n'

In [2]:
atoms = read('Au_particle.xyz')

In [3]:
atoms

Atoms(symbols='C144H162Au25S18', pbc=False)

In [6]:
import torch
import pickle

# Load the saved DataLoader
filename = 'dataloader.pkl'
with open(filename, 'rb') as f:
    loaded_dataloader = pickle.load(f)

In [7]:
#atoms = read('Au_particle.xyz')
spookyNet_model = 'prueba.pth'
charge = 1
magmom = 0 
nbeads = 10
use_gpu = True

In [8]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060'

In [9]:
# load SpookyNet model
try:
    model = SpookyNet(load_from=spookyNet_model)
    model.to(torch.float32)
    model.eval()
    print(
        " @ForceField: SpookyNet model " + spookyNet_model + " loaded"
    )
except ValueError:
    raise ValueError(
        "ERROR: Reading SpookyNet model " + spookyNet_model + " file failed."
    )

if use_gpu and not torch.cuda.is_available():
    print(
        " @ForceField: No GPU available: Evaluation on GPU was requested"
        + " but no GPU device is available. Falling back to CPU."
    )

if use_gpu and torch.cuda.is_available():
    model.cuda()
    device = 'cuda:0'
    
else:
    device = 'cpu'
print(device)

 @ForceField: SpookyNet model prueba.pth loaded
cuda:0


In [10]:
enegias = []

In [17]:
j = 0
for batch in loaded_dataloader:
    # Process the batch as needed
    datos = batch  # Unpack the batch into inputs and targets
    nxyz = datos['nxyz']
    xyz = nxyz[:, 1:]
    xyz = xyz.numpy()
    len_atoms = nxyz.shape[0]
    nu_atoms = nxyz[:, 0]

    nu = datos['num_atoms']
    # generate idx lists for finding neighboring atoms
    idx = torch.arange(len_atoms, dtype=torch.int64, device=device)
    idx_i = idx.view(-1, 1).expand(-1, len_atoms).reshape(-1)
    idx_j = idx.view(1, -1).expand(len_atoms, -1).reshape(-1)

    idx_i = datos['mol_nbrs'][:, 0].to(device)
    idx_j = datos['mol_nbrs'][:, 1].to(device)

    # exclude self-interactions
    idx_i, idx_j = idx_i[idx_i != idx_j], idx_j[idx_i != idx_j]

    batch_seg = torch.cat([torch.ones(int(num_atoms)) * i for i, num_atoms in
                               enumerate(nu)]).to(torch.long).to(device)
    # create input dictionary
    inputs2 = {
        "Z": torch.tensor(nu_atoms, dtype=torch.int64, device=device),
        "Q": torch.full((nbeads,), charge, dtype=torch.float32, device=device),
        "S": torch.full((nbeads,), magmom, dtype=torch.float32, device=device),
        "idx_i": torch.cat([idx_i + i * len_atoms for i in range(1)]),
        "idx_j": torch.cat([idx_j + i * len_atoms for i in range(1)]),
        "batch_seg": batch_seg,
        "num_batch": 10
    }
    R = torch.tensor(xyz,
                    dtype=torch.float32,
                    device=device)
    R.requires_grad = True
    inputs2["R"] = R

    outputs = model.energy_and_forces(**inputs2)
    energy = outputs[0].detach().cpu().numpy()
    forces = outputs[1].detach().cpu().numpy().reshape((1, -1, 3))
    enegias.append(energy)


C:\Users\fraja\AppData\Local\Temp\ipykernel_23304\2800878389.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "Z": torch.tensor(nu_atoms, dtype=torch.int64, device=device),


------------------Zzzzzzzzzzzzzzzzzzzzzz torch.Size([170]) ------------------Qqqqqqqqqqqqqqq torch.Size([10]) ------------------Sssssssssssss torch.Size([10]) ------------------R torch.Size([170, 3]) ------------------idx_i torch.Size([1360]) ------------------idx_j torch.Size([1360]) ------------------cell None ------------------cell_offsets None ------------------num_batch 10 ------------------batch_seg tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
        5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7,
        7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8,
        8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9,
        9, 9], device='cuda:0')